# Сбор постов из групп VK

In [1]:
import os
import sys
import json
from dotenv import dotenv_values

In [2]:
root_dir = os.path.join(os.getcwd(), os.path.pardir)
os.chdir(root_dir)

In [3]:
env = dotenv_values()

group_id_file_path = os.path.join(root_dir,
                                  'notebooks',
                                  'group_to_collect_posts.json')

with open(group_id_file_path, 'r') as group_id_file:
    group_id = json.load(group_id_file)['target_group_id']

## Собираем посты

In [14]:
from pprint import pprint
from tqdm.notebook import tqdm
from src.vk.posts_getter import PostsGetter
from src.vk.posts_handler import PostsHandler


POSTS_TO_COLLECT = 1000
OFFSET_STEP = 50

collected_posts = []

posts_getter = PostsGetter(env['VK_ACCESS_TOKEN'])
group_name = posts_getter.get_group_name(group_id)
posts_handler = PostsHandler()

for offset in tqdm(range(0, POSTS_TO_COLLECT, OFFSET_STEP)):
    raw_posts = posts_getter.get_posts_data(group_id, OFFSET_STEP, offset)
    posts = posts_handler.process_posts_data(raw_posts, group_name)
    collected_posts.extend(posts)

  0%|          | 0/20 [00:00<?, ?it/s]

## Сохраняем тексты постов

In [71]:
for counter, post in enumerate(collected_posts):
    fname_to_save = os.path.join('texts', f'{counter}.txt')
    with open(fname_to_save, 'w') as text_file:
        text_file.write(post.text)

## Чистим тексты постов

In [63]:
import re


def clean_text(text):
    clear_text = re.sub(r'http\S+', '', text)
    clear_text = re.sub(r'#\S+', '', clear_text)
    clear_text = re.sub('[^A-Za-zА-Яа-я]+', ' ', clear_text)
    clear_text = clear_text.lower()
    clear_text = clear_text.strip()
    return clear_text

clear_texts = [clean_text(post.text) for post in collected_posts]

In [64]:
print('ОРИГИНАЛЬНЫЙ ТЕКСТ ПОСТА', end='\n'*2)
print(collected_posts[3].text)

ОРИГИНАЛЬНЫЙ ТЕКСТ ПОСТА

◆ЗАМЕНА ОСТЕКЛЕНИЯ◆ 
НА БАЛКОНАХ И ЛОДЖИЯХ 
В СПб и ЛО от "ПРЕМИУМ БАЛКОН" 

✔ Выполняют весь комплекс работ - замену остекления/остекление, утепление, изоляцию, вывод электрики, отделку. 
✔ Предоставляют возможность поэтапной оплаты. 
✔ Дают гарантии на работы до 3 лет. 
✔ Являются официальными дилерами Rehau 

Работают "ПОД КЛЮЧ" 
✔ От расчёта стоимости при первом обращении, до выноса мусора по окончанию работ. 

Имеют большое ПОРТФОЛИО 
✔Ежегодно создают более 5000 теплых балконов и лоджий из холодных технических помещений. Посмотреть портфолио можно по ссылке: https://vk.com/albums-131675173 

Хотите себе идеальную стильную комнату вместо холодной лоджии? 

Оставить заявку можно по следующим контактам: 

✅ Вконтакте: https://vk.com/osteklenie_balkona_spb 
✅ Телефоны компании: 📞 +7 (921) 903-74-72 и +7 (812) 775-60-58 
✅ Официальный сайт: https://premiumbalkon.ru 

Запишитесь на бесплатный замер прямо сейчас! 

#остеклениелоджий #остеклениебалкона #отделкас

In [65]:
print('ОЧИЩЕННЫЙ ТЕКСТ ПОСТА', end='\n'*2)
print(clear_texts[3])

ОЧИЩЕННЫЙ ТЕКСТ ПОСТА

замена остекления на балконах и лоджиях в спб и ло от премиум балкон выполняют весь комплекс работ замену остекления остекление утепление изоляцию вывод электрики отделку предоставляют возможность поэтапной оплаты дают гарантии на работы до лет являются официальными дилерами rehau работают под ключ от расч та стоимости при первом обращении до выноса мусора по окончанию работ имеют большое портфолио ежегодно создают более теплых балконов и лоджий из холодных технических помещений посмотреть портфолио можно по ссылке хотите себе идеальную стильную комнату вместо холодной лоджии оставить заявку можно по следующим контактам вконтакте телефоны компании и официальный сайт запишитесь на бесплатный замер прямо сейчас


In [68]:
texts = '\n'.join(clear_texts) # тексты разделены символами новой строки

with open('collected_texts.txt', 'w') as texts_file:
    texts_file.write(texts)